## Chargement des fichiers

In [1]:
import pandas as pd
import re
import requests
import os
from ipywidgets import IntProgress
from IPython.display import display
from pymarc import parse_xml_to_array
import jellyfish
from datetime import datetime

pd.options.mode.chained_assignment = None  # Remove copy in place warning for dataframe operations


# Définition du fichier d'alignement à vérifier
input_file_path = "input/verif-auto-102024/rapport_20241030151058_alignement-défini_TEST Mat personnes_Tout.tsv"

# Définition du fichier de sortie pour les cas suspects
output_file_root = "output/verif-auto-102024/verif-auto-personnes-Mat"

# Faut-il exporter tous les alignements vérifiés avec une colonne identifiant les cas suspects (True),
# ou exporter ces derniers à part (False)?
output_in_place = True

# Types d'alignement à vérifier
valid_types = {'auto'}

# Types de validation à effectuer
validations = {'idref-local', 'dates', 'professions', 'similarité', 'auteur-titre'}
# Les valeurs suivantes sont possibles:
# - 'dates' -> compare les dates présentes dans les formes principales source et cible. PEUT être combiné avec 'idref-local'
# - 'auteur-titre' ATTENTION cette validation fait un appel d'API IdRef. Utiliser uniquement lorsque nécessaire.
# - 'idref-local' -> compare les sous-champs des notices source et cible. Nécessite l'accès à un export des notices IdRef et ATC/RNV
# - 'professions' -> compare les sous-champs $c si présents. DOIT être combiné avec 'idref-local'
# - 'similarité' -> tente de calculer un coéfficient de similarité entre les formes principales. PEUT être combiné avec 'idref-local'

# Définition des fichiers exportés de IdRef et ATC/RNV pour comparaison des sous-champs. Utilisé uniquement en mode 'idref-local'
idref_set = 'personnes'
idref_records_folder = "/Users/thomas/Documents/tmp-nobackup/bcu-rnv/idref-harvest-" + idref_set
atc_sets = ['personnes']
atc_records_file_paths = ["input/notices-detaillees/20241028_auth_mat_personnes_subf_a-d.tsv"]

# Pour limiter le chargement des fichiers IdRef (ex. pour tester), renseigner cette variable pour ne charger que les x premier fichiers.
# Pour ne pas limiter, assigner la valeur -1
idref_records_limit = -1

# Fichiers temporaires pour accélérer le traitement successif lors de la comparaison des sous-champs. Si existants, ils sont utilisés
# en mode 'idref-local' en lieu et place des notices complètes ci-dessus.
idref_records_extract = "input/idref-extrait-" + idref_set + "-comparaison.csv"

# Nom de la colonne ajoutée
output_colname = "validation auto"

# Lors des comparaisons textuelles, la distance calculée sera renseignée dans cette colonne
distance_colname = "distance validation"

# Définit la valeur de distance (rapportée à la longueur de la forme source) à partir de laquelle on considère qu'il y a une différence.
distance_limit = 0.15

# Ajoute une colonne avec des détails de comparaison, pour débogger
debug_column = True

# Définition des colonnes contenant les formes sources et cible à comparer
source_form_colname = "forme principale source"
target_form_colname = "forme principale cible"
align_type_colname = "décision d'alignement"
target_colname = "réservoir cible"
target_candidatenr_colname = "nombre de candidats"
idref_id_colname = "id cible"
atc_id_colname = "id source"

# Vérifier que tous les fichiers existent vraiment avant de continuer
all_files = []
for variable in [input_file_path, atc_records_file_paths]:
    if isinstance(variable, list):
        all_files.extend(variable)
    else:
        all_files.append(variable)

for file in all_files:
    if file and not os.path.isfile(file):
        print("ATTENTION le fichier suivant n'existe pas: ",file)

# Le fichier à vérifier est chargé dans une dataframe
df = pd.read_csv(input_file_path, sep='\t', dtype = str)
print(df.shape)

# Pour la comparaison, on retire les types d'alignements ignorés ainsi que les non-alignements
df_filtered = df[(df[align_type_colname].isin(valid_types)) & (df[target_colname] == 'idref')]
df_rest = df[(~df[align_type_colname].isin(valid_types)) | (df[target_colname] != 'idref')]
print(df_filtered.shape)
print(df_rest.shape)
display(df_filtered.head())

df_filtered.reset_index(drop=True, inplace=True)  # Drop indexes since we're not going to use them

(19195, 23)
(10416, 23)
(8779, 23)


,réservoir source,id source,forme principale source,arbitre,date d'arbitrage,niveau de confiance,commentaire,décision d'alignement,nombre de candidats,score algo max,...,id cible,forme principale cible,type de cible 2,réservoir cible 2,id cible 2,forme principale cible 2,type de cible 3,réservoir cible 3,id cible 3,forme principale cible 3
5,rnv,009950613,"Moussorgski, Modeste",NaN,NaN,NaN,NaN,auto,1,0.9742324561403508,...,050121960,"Musorgskij, Modest Petrovič 1839-1881",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,rnv,009950616,"Auric, Georges",NaN,NaN,NaN,NaN,auto,1,1.0,...,02807808X,"Auric, Georges 1899-1983",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,rnv,009950619,"Milhaud, Darius",NaN,NaN,NaN,NaN,auto,1,1.0,...,027029549,"Milhaud, Darius 1892-1974",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,rnv,009950620,"Poulenc, Francis",NaN,NaN,NaN,NaN,auto,1,1.0,...,030260655,"Poulenc, Francis 1899-1963",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,rnv,009950621,"Tailleferre, Germaine",NaN,NaN,NaN,NaN,auto,1,1.0,...,033067414,"Tailleferre, Germaine 1892-1983",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Chargement des fichiers auxiliaires

Si une comparaison utilisant les notices complètes IdRef et ATC/RNV est demandée, charger ces dernières dans un dataframe.

In [2]:
# Fonctions nécessaires à traiter les fichiers XML provenant d'IdRef (Marc21)
tps_debut_chargement = datetime.now()

# Chargement des fichiers individuels
def parse_marcxml_file(file_path):
    records = []
    with open(file_path, 'rb') as fh:
        marc_records = parse_xml_to_array(fh)
        for record in marc_records:
            if record != None:
                record_dict = extract_record_data(record)
                if record_dict != None:
                    records.append(record_dict)
    return records

# Extraction des champs nécessaires
def extract_record_data(record):
    field_001 = record.get('001')
    if idref_set == 'personnes':
        field_100 = record.get('100')
        if field_001:
            record_data = {
                'idref_id' : field_001.data.replace('(IDREF)','').strip(),
                '100a': field_100.get('a') if field_100 else None,
                '100b': field_100.get('b') if field_100 else None,
                '100c': field_100.get('c') if field_100 else None,
                '100d': field_100.get('d') if field_100 else None
            }
        else:
            return None
    elif idref_set == 'collectivités':
        field_110 = record.get('110')
        field_111 = record.get('111')
        field_411 = record.get('411')
        if field_001:
            record_data = {
                'idref_id' : field_001.data.replace('(IDREF)','').strip(),
                '110a': field_110.get('a') if field_110 else None,
                '110b': field_110.get('b') if field_110 else None,
                '110g': field_110.get('g') if field_110 else None,
                '111a': field_111.get('a') if field_111 else None,
                '111c': field_111.get('c') if field_111 else None,
                '111d': field_111.get('d') if field_111 else None,
                '111n': field_111.get('n') if field_111 else None,
                '411a': field_111.get('a') if field_411 else None,
                '411c': field_111.get('c') if field_411 else None,
                '411d': field_111.get('d') if field_411 else None,
                '411n': field_111.get('n') if field_411 else None
            }
    return record_data

# Chargement des fichiers du dossier IdRef
def load_marcxml_files_from_folder(folder_path, limit, status_display):
    all_records = []
    
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.xml'): 
            file_path = os.path.join(folder_path, file_name)
            records = parse_marcxml_file(file_path)
            all_records.extend(records)
        status_display.value += 1
        if (limit > 0 and status_display.value == limit):
            break
    
    # Convert list of records to DataFrame
    df = pd.DataFrame(all_records)
    return df

# Chargement des fichiers auxiliaires si on a choisi le type de validation 'idref-local'
if 'idref-local' in validations:
    
    # Si un fichier CSV a été fourni (l'extraction a déjà été faite), charger les notices depuis ce fichier
    if idref_records_extract and os.path.isfile(idref_records_extract):
        idRef_df = pd.read_csv(idref_records_extract, dtype = str)
        print(f"{len(idRef_df.index)} notices IdRef chargées depuis {idref_records_extract}")
    else:
        # Sinon, on prend une grande respiration et on charge tous les fichiers IdRef dans un dataframe
        
        # Affiche une barre de progression
        if idref_records_limit > 0:
            num_files = idref_records_limit
        else:
            num_files = sum(1 for entry in os.scandir(idref_records_folder) if entry.is_file() and entry.name.endswith('.xml'))
        barre_attente_idRef = IntProgress(min=1, max=num_files, description="Chargement des fichiers IdRef: ", style={'description_width':'initial'},layout={'width':'80%'})
        display(barre_attente_idRef)
        
        idRef_df = load_marcxml_files_from_folder(idref_records_folder, idref_records_limit, barre_attente_idRef)
        
        # Une fois le chargement terminé, on écrit le résultat dans un fichier CSV
        if idref_records_extract:
            idRef_df.to_csv(idref_records_extract, encoding="UTF-8", index=False)
    
        print(f"{len(idRef_df.index)} notices IdRef chargées depuis export OAI IdRef dans {idref_records_folder}")
    
    # Utiliser l'identifiant IdRef comme index (pour accélérer les requêtes)
    idRef_df.set_index('idref_id', inplace=True)
    # Retirer les éventuels doublons
    idRef_df = idRef_df[~idRef_df.index.duplicated(keep='first')]
    
    # Chargement des fichiers ATC/RNV
    atc_df_list = []
    for file_index, atc_records_file_path in enumerate(atc_records_file_paths):
        if atc_records_file_path and os.path.isfile(atc_records_file_path):
            atc_df_toadd = pd.read_csv(atc_records_file_path, sep='\t', dtype = str)
            atc_set = atc_sets[file_index]
            # Extraire le contenu des sous-champs dans des colonnes distinctes (en enlevant les dernières virgules, si présentes)
            if atc_set == 'personnes':
                atc_df_toadd['100a'] = atc_df_toadd['subfields_content_for_tag2'].str.extract(r'\$\$a ([^$]+)')[0].str.replace(r'[,\s]+$', '', regex=True)
                atc_df_toadd['100b'] = atc_df_toadd['subfields_content_for_tag2'].str.extract(r'\$\$b ([^$]+)')[0].str.replace(r'[,\s]+$', '', regex=True)
                atc_df_toadd['100c'] = atc_df_toadd['subfields_content_for_tag2'].str.extract(r'\$\$c ([^$]+)')[0].str.replace(r'[,\s]+$', '', regex=True)
                atc_df_toadd['100d'] = atc_df_toadd['subfields_content_for_tag2'].str.extract(r'\$\$d ([^$]+)')[0].str.replace(r'[,\s]+$', '', regex=True)
            elif atc_set == 'collectivités':
                atc_df_toadd['110a'] = atc_df_toadd['subfields_content_for_tag2'].str.extract(r'\$\$a ([^$]+)')[0].str.replace(r'[,\s]+$', '', regex=True)
                atc_df_toadd['110b'] = atc_df_toadd['subfields_content_for_tag2'].str.extract(r'\$\$b ([^$]+)')[0].str.replace(r'[,\s]+$', '', regex=True)
                atc_df_toadd['110g'] = atc_df_toadd['subfields_content_for_tag2'].str.extract(r'\$\$g ([^$]+)')[0].str.replace(r'[,\s]+$', '', regex=True)
            elif atc_set == 'congrès':
                atc_df_toadd['111a'] = atc_df_toadd['subfields_content_for_tag2'].str.extract(r'\$\$a ([^$]+)')[0].str.replace(r'[,\s]+$', '', regex=True)
                atc_df_toadd['111c'] = atc_df_toadd['subfields_content_for_tag2'].str.extract(r'\$\$c ([^$]+)')[0].str.replace(r'[,\s]+$|\)', '', regex=True).str.strip()
                atc_df_toadd['111d'] = atc_df_toadd['subfields_content_for_tag2'].str.extract(r'\$\$d ([^$]+)')[0].str.replace(r'[,\s]+$|\(|:', '', regex=True).str.strip()
                atc_df_toadd['111n'] = atc_df_toadd['subfields_content_for_tag2'].str.extract(r'\$\$n ([^$]+)')[0].str.replace(r'[,\s]+$|\(|:', '', regex=True).str.strip()
            atc_df_list.append(atc_df_toadd)
            print(f"{len(atc_df_toadd.index)} notices ATC/RNV chargées depuis le fichier {atc_records_file_path}")
        else:
            print('Attention! Le fichier des notices ATC/RNV est manquant!')
    atc_df = pd.concat(atc_df_list)
    # Utiliser l'identifiant comme index (pour accélérer les requêtes)
    atc_df.set_index('id', inplace=True)
    print(f"{len(atc_df.index)} notices ATC/RNV chargées au total.")

tps_fin_chargement = datetime.now()
print("--- Temps écoulé pour le chargement des fichiers: %s ---" % format(tps_fin_chargement - tps_debut_chargement))


4293697 notices IdRef chargées depuis input/idref-extrait-personnes-comparaison.csv
951 notices ATC/RNV chargées depuis le fichier input/notices-detaillees/20241028_auth_mat_personnes_subf_a-d.tsv
951 notices ATC/RNV chargées au total.
--- Temps écoulé pour le chargement des fichiers: 0:00:10.449188 ---


## Définition des fonctions utiles

In [3]:
# Fonction de comparaison des dates
def date_compare(source_date,target_date):
    source_date = str(source_date)
    target_date = str(target_date)
    # Ignorer les points d'interrogation
    source_date = source_date.replace('?','')
    target_date = target_date.replace('?','')
    # Retirer le zéro en début de date
    if source_date.startswith('0'):
        source_date = source_date[1:0]
    if target_date.startswith('0'):
        target_date = target_date[1:0]
    # Si l'une des dates à comparer est vide, on passe
    if ((source_date == '') | (target_date == '')):
        return False
    # Si l'une des dates comporte des points, ne comparer que les chiffres entre eux
    if (('.' in source_date) | ('.' in target_date)):
        #print('Comparaison avec points: ' + source_date + ' et ' + target_date)
        for char in range(0,len(source_date)):
            if ((source_date[char] != '.') & (target_date[char] != '.') & (source_date[char] != target_date[char])):
                #print('Je pense que ' + source_date[char] + ' != ' + target_date[char])
                return True
        return False
    if (source_date != target_date):
        return True
    else:
        return False

# Expression régulière pour trouver les dates
date_pattern = r'(?:ca\.|fl\.)?([0-9\.]{4}|\?)-?([0-9\.]{4}|\?)?'

# Vérification des auteurs-titre

idref_base_url = 'https://idref.fr/'

# Fonction utile pour déterminer si un champ MARC existe
def contains_tag(data, tag_value):
    return any(record['tag'] == tag_value for record in data)



## Moulinette de validation
C'est là que tout se passe!

In [4]:
# Affiche une barre de progression
numrows = df_filtered.shape[0]
barre_attente = IntProgress(min=1, max=numrows, description="État d'avancement: ", style={'description_width':'initial'},layout={'width':'80%'})
display(barre_attente)

if not(output_in_place):
    wrong_dates = []
    wrong_types = []
    wrong_forms = []
    misc_errors = []

nb_api_calls = 0

tps_debut_moulinette = datetime.now()

for index, row in df_filtered.iterrows():
    if not ((row[source_form_colname] == '') | (row[target_form_colname] == '') | pd.isnull(row[source_form_colname]) | pd.isnull(row[target_form_colname])):

        source_dates = ''
        target_dates = ''
        verif_dates = False

        source_term = ''
        target_term = ''

        notice_idref = None
        notice_atc = None

        distance = None

        debug_output = ''
        
        # Procéder à la validation en utilisant les notices complètes IdRef et ATC/RNV si demandé
        
        if 'idref-local' in validations:
            # La validation ne peut se faire que si les notices complètes existent
            try:
                notice_idref = idRef_df.loc[row[idref_id_colname]]
            except (KeyError, TypeError) as e:
                # Ignorer cette ligne s'il manque une des données requises
                pass
            try:
                notice_atc = atc_df.loc[row[atc_id_colname]]
            except (KeyError, TypeError) as e:
                # Ignorer cette ligne s'il manque une des données requises
                pass
            if 'dates' in validations:
                try:
                    source_dates = re.findall(date_pattern, notice_atc['100d'])
                except (KeyError, TypeError) as e:
                    # Ignorer cette ligne s'il manque une des données requises
                    pass
                try:
                    target_dates = re.findall(date_pattern, notice_idref['100d'])
                except (KeyError, TypeError) as e:
                    # Ignorer cette ligne s'il manque une des données requises
                    pass
            if 'similarité' in validations:
                try:
                    source_term = notice_atc['100a']
                except (KeyError, TypeError) as e:
                    # Ignorer cette ligne s'il manque une des données requises
                    pass
                try:
                    target_term = notice_idref['100a']
                except (KeyError, TypeError) as e:
                    # Ignorer cette ligne s'il manque une des données requises
                    pass
            if 'professions' in validations:
                try:
                    source_prof = notice_atc['100c']
                    target_prof = notice_idref['100c']
                    distance_prof = jellyfish.levenshtein_distance(source_prof,target_prof)/len(source_prof)
                    if distance_prof > distance_limit:
                        if (output_in_place):
                            df_filtered.loc[index, output_colname] = "Vérifier professions"
                            df_filtered.loc[index, distance_colname] = distance_prof
                    elif (output_in_place):
                            df_filtered.loc[index, output_colname] = "OK ($$c)"
                except (KeyError, TypeError) as e:
                    # Ignorer cette validation s'il manque une des données requises
                    pass
        
        # Extraire les dates de la forme principale si on ne les a pas trouvées dans les sous-champs'
        if len(source_dates) < 1:
            source_dates = re.findall(date_pattern, row[source_form_colname])
        if len(target_dates) < 1:
            target_dates = re.findall(date_pattern, row[target_form_colname])

        if debug_column:
            debug_output = debug_output + f"Dates {source_dates} et {target_dates} | "
        
        # Validation de date si elles sont présentes des deux côtés et si demandé
        if ((len(source_dates) > 0) and (len(target_dates) > 0) and ('dates' in validations)):
            if (date_compare(source_dates[0][0],target_dates[0][0]) | date_compare(source_dates[0][1],target_dates[0][1])):
                # Cas potentiel de dates qui ne correspondent pas
                if (output_in_place):
                    df_filtered.loc[index, output_colname] = "Vérifier dates"
                else:
                    wrong_dates.append(row)
            else:
                # Les dates semblent correspondre, utiliser cette information pour valider l'alignement
                verif_dates = True
                df_filtered.loc[index, output_colname] = "OK (dates)"

        
        # Utiliser les formes simples pour la similarité textuelle (sans chiffres et signes de ponctuation) si on n'a pas pu extraire le sous-champ $a
        if len(source_term) < 1:
            source_term = re.sub(r'[\d,-.\?()]', '', row[source_form_colname]).strip()

        if len(target_term) < 1:
            target_term = re.sub(r'[\d,-.\?()]', '', row[target_form_colname]).strip()

        if debug_column:
            debug_output = debug_output + f"Comparaison entre {source_term} et {target_term} | "
        
        # Calculer la similarité textuelle si demandé
        if (len(source_term) > 0) and (len(target_term) > 0) and ('similarité' in validations):
            distance = jellyfish.levenshtein_distance(source_term,target_term)/len(source_term)
            if debug_column:
                debug_output = debug_output + f"distance: {str(distance)} | "

        if (distance is not None and distance > distance_limit and not(verif_dates)):
            # Signaler comme erreur potentielle sauf si validé par les dates
            if (output_in_place):
                df_filtered.loc[index, output_colname] = "Vérifier forme principale"
                df_filtered.loc[index, distance_colname] = distance
            else:
                wrong_forms.append(row)
        elif (distance is not None and distance <= distance_limit and not(verif_dates) and output_in_place):
                df_filtered.loc[index, output_colname] = "OK (levenshtein)"
                df_filtered.loc[index, distance_colname] = distance

        
        # Vérifier si la notice IdRef vers laquelle on aligne est de type auteur-titre, seulement si demandé
        if 'auteur-titre' in validations:
            if notice_idref is None:
                # Si la notice cible n'est pas disponible localement, il ne s'agit probablement pas d'une notice de type auteur
                # Faire un appel d'API pour en avoir le coeur net.
                # ATTENTION cette étape est très coûteuse en temps
                idref_id = row[idref_id_colname]
                url = idref_base_url + str(idref_id) + '.json'
                #print("Appel d'API pour " + str(idref_id))
                nb_api_calls += 1
                # Appel d'API
                idref_result = requests.get(url)
                if (idref_result.status_code == 200):
                    # Identifier si le résultat contient un champ 240
                    if contains_tag(idref_result.json()['record']['datafield'], 240):
                        if (output_in_place):
                            df_filtered.loc[index, output_colname] = "Cible de type auteur-titre"
                        else:
                            wrong_types.append(row)
                else:
                    # Erreur de requête API
                    if (output_in_place):
                        df_filtered.loc[index, output_colname] = "Erreur de requête API IdRef. Code:" + str(idref_result.status_code)
                    else:
                        misc_errors.append(row)
    
    else:
        # Il manque un des champs à comparer, autre erreur
        if (output_in_place):
            df_filtered.loc[index, output_colname] = "Erreur de validation"
        else:
            misc_errors.append(row)
    
    if debug_column:
        df_filtered.loc[index, "debug"] = debug_output
    
    barre_attente.value += 1

display(df_filtered.head())

tps_fin_moulinette = datetime.now()
print("Nombre d'appels d'API: " + str(nb_api_calls) + " (" + str(round(nb_api_calls/numrows*100)) + "% des alignements)")
print("--- Temps écoulé pour la comparaison des alignements: %s ---" % format(tps_fin_moulinette - tps_debut_moulinette))

IntProgress(value=1, description="État d'avancement: ", layout=Layout(width='80%'), max=10416, min=1, style=Pr…

,réservoir source,id source,forme principale source,arbitre,date d'arbitrage,niveau de confiance,commentaire,décision d'alignement,nombre de candidats,score algo max,...,réservoir cible 2,id cible 2,forme principale cible 2,type de cible 3,réservoir cible 3,id cible 3,forme principale cible 3,validation auto,distance validation,debug
0,rnv,009950613,"Moussorgski, Modeste",NaN,NaN,NaN,NaN,auto,1,0.9742324561403508,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vérifier forme principale,0.631579,"Dates [] et [('1839', '1881')] | Comparaison e..."
1,rnv,009950616,"Auric, Georges",NaN,NaN,NaN,NaN,auto,1,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OK (levenshtein),0.076923,"Dates [] et [('1899', '1983')] | Comparaison e..."
2,rnv,009950619,"Milhaud, Darius",NaN,NaN,NaN,NaN,auto,1,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OK (levenshtein),0.071429,"Dates [] et [('1892', '1974')] | Comparaison e..."
3,rnv,009950620,"Poulenc, Francis",NaN,NaN,NaN,NaN,auto,1,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OK (levenshtein),0.066667,"Dates [] et [('1899', '1963')] | Comparaison e..."
4,rnv,009950621,"Tailleferre, Germaine",NaN,NaN,NaN,NaN,auto,1,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OK (levenshtein),0.050000,"Dates [] et [('1892', '1983')] | Comparaison e..."


Nombre d'appels d'API: 779 (7% des alignements)
--- Temps écoulé pour la comparaison des alignements: 0:28:31.757764 ---


## Exports des fichiers de résultats

In [5]:
tps_debut_sauvegarde = datetime.now()

if not(output_in_place):
    wrong_dates_df = pd.DataFrame(wrong_dates)
    wrong_forms_df = pd.DataFrame(wrong_forms)
    misc_errors_df = pd.DataFrame(misc_errors)

    wrong_dates_df.to_excel(output_file_root + "_dates.xlsx",index=False)
    wrong_forms_df.to_excel(output_file_root + "_formes.xlsx",index=False)
    misc_errors_df.to_excel(output_file_root + "_erreurs.xlsx",index=False)
else:
    print(df_filtered.shape)
    # Combiner le fichier filtré qui a été validé avec les alignements retirés au début
    df_output = pd.concat([df_filtered,df_rest])
    print(df_output.shape)
    df_output.to_excel(output_file_root + "_tout.xlsx",index=False)

tps_fin_sauvegarde = datetime.now()
print("--- Temps écoulé pour sauvegarder les fichiers: %s ---" % format(tps_fin_sauvegarde - tps_debut_sauvegarde))
print("--- Temps total écoulé: %s ---" % format(tps_fin_sauvegarde - tps_debut_chargement))

(10416, 26)
(19195, 26)
--- Temps écoulé pour sauvegarder les fichiers: 0:00:09.097654 ---
--- Temps total écoulé: 0:28:51.345674 ---
